In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd 
data = pd.read_csv('/content/drive/MyDrive/geekstest1.csv',encoding = "ISO-8859-1")  
data = data.drop(['url','rating'],axis=1)
data

,title,content,tags
0,Python | Thresholding techniques using OpenCV ...,Prerequisite: Simple Thresholding using OpenC...,"{'Python', 'OpenCV', 'Image-Processing'}"
1,Generation of n numbers with given set of factors,"Given an array of k numbers factor[], the task...","{'prime-factor', 'Mathematical'}"
2,C | Operators | Question 17,Which of the following can have different mean...,"{'C', 'C-Operators', 'Operators', 'C Quiz'}"
3,SymPy | Subset.size() in Python,Subset.size() : size() is a sympy Python libra...,{'Python'}
4,Python PIL | ImageFont.truetype(),PIL is the Python Imaging Library which provid...,"{'Python', 'Python-pil'}"
...,...,...,...
24484,Importance of Hashcode method in Java,Prerequisite: Equals() and hashCode() methods ...,"{'Java', 'java-hashset', 'Java-HashMap'}"
24485,Ruby | Queue close() function,The close() is an inbuilt function in Ruby cl...,"{'Ruby Collections', 'Ruby'}"
24486,Scala | Field Overriding,"In any object-oriented programming language, O...","{'Scala', 'Scala-OOPS'}"
24487,Sum of minimum elements of all possible sub-ar...,"Given an array arr[], the task is to find the ...","{'C++ Programs', 'Data Structures', 'Stack', '..."


In [ ]:
corpus = []
titletoind_dict = {} #mapping of conversation id with corpus index
j = 0
for sub,body in zip(data['title'],data['content']):
    if body is None or body is '':
        corpus += [(sub)]
    else:
        corpus += [(sub+" "+body)]
    titletoind_dict[j] = sub
    j += 1
len(corpus)

24489

In [ ]:
!pip install requests
!pip install spacy
!pip install MySQL
!pip install mysql-connector-python
!pip install MySQL-python
!python -m spacy download en
!pip install --user scikit-learn

  Using cached https://files.pythonhosted.org/packages/a5/e9/51b544da85a36a68debe7a7091f068d802fc515a3a202652828c73453cad/MySQL-python-1.2.5.zip
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
import spacy 
import string,re
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer(language='english')

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[\!\"\#\$\%\&\(\)\*\+\,\-\/\:\;\<\=\>\?\@\^\`\{\}\|\~\\\]')
REPLACE_BY_NULL_RE = re.compile('[\']')
ver = '[0-9]+\.[0-9]+\.*[0-9]*' #regex that identifies version number 
dot = re.compile('[/.]') #identifies '.' in tokens
fs = '[a-z]*\.[a-z]*' #regex that identifies words having '.'
cr_pattern = re.compile(pattern = "["u'\N{COPYRIGHT SIGN} \N{TRADE MARK SIGN} \N{REGISTERED SIGN}'"]+", flags = re.UNICODE)#remove copyright symbols
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)
def preprocess(body):
    if body is None:
        return None
    body = body.lower()
    body = deEmojify(body)
    body = REPLACE_BY_SPACE_RE.sub(' ', body)
    body = cr_pattern.sub(' ',body)
    l = []
    for token in body.split(" "):
        if(re.search(ver,token)):
            token = token
        elif(re.search(fs,token)):
            l += [dot.sub(' ',token)]
            continue
        l += [token]
    body = ' '.join(token for token in l)
    body = re.sub('\s+',' ',body)             
    body = body.strip()
    return body

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner']) #pos tagging + lammetization
nlp.max_length = 3000000
def pos_tag(corpus):
    corpus = (" ".join([token.lemma_ for token in nlp(corpus) if token.pos_ in ('NOUN','VERB','ADJ','ADV','NUM')]))
    return corpus

In [ ]:
for i in range(len(corpus)):
    corpus[i] = preprocess(corpus[i])
    corpus[i] = pos_tag(corpus[i])
    corpus[i] = REPLACE_BY_NULL_RE.sub('', corpus[i])   #removing \' incase if it didn't get eliminated in lammetization
    corpus[i] = remove_stopwords(corpus[i])             #removing stopwords
    n_st = " "
    corpus[i] = n_st.join(stemmer.stem(token) for token in corpus[i].split()) #stemming

In [ ]:
print(i)

24488


In [ ]:
dataset = [d.split() for d in corpus] #form bigrams and trigrams
bigram = gensim.models.phrases.Phrases(dataset, min_count=10, threshold=20.0)
trigram = gensim.models.phrases.Phrases(bigram[dataset], min_count=10, threshold=20.0)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
from collections import Counter
l1,l2 = [],[]
reg = '\w+\_\w+'        #regex to identify a bigram
rege = '\w+\_\w+\_\w+'  #regex to identify a trigram
for i in range(len(dataset)):
    for token in trigram[bigram[dataset[i]]]:
        if (re.search(reg,token)):
            l1 += [token]
        if (re.search(rege,token)):
            l2 += [token]
Counter1 = Counter(l1)
most_occur = Counter1.most_common(40)    #calculates 40 common bigrams
print(most_occur)
Counter = Counter(l2)
most_occuring = Counter.most_common(40)  #calculates 40 common trigrams
print(most_occuring)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[('comment_incorrect_want_share', 5240), ('inform_topic_discuss', 5238), ('strong_recommend_practic', 2549), ('syntax_paramet', 1484), ('solv_problem', 968), ('binari_tree', 752), ('leav_right', 722), ('simpl_solut', 669), ('bad_case', 521), ('extra_space', 485), ('problem_solv', 461), ('prime_factor', 414), ('hash_tabl', 393), ('discuss_certain_way', 324), ('short_path', 275), ('regular_express', 274), ('header_file', 254), ('ascend_order', 247), ('datum_structur', 247), ('nth_term', 247), ('quiz_question', 234), ('leaf_node', 227), ('binari_search', 224), ('file_descriptor', 216), ('mention_div_append_interview', 203), ('local_variabl', 187), ('previous_post', 184), ('binari_search_tree', 184), ('php_implement_approach', 182), ('csv_file', 180), ('littl_bit_tricki', 173), ('descend_order', 172), ('32_bit', 171), ('perfect_squar', 165), ('oper_come_standard', 161), ('binari_repres', 154), ('public_static_void', 146), ('throw_nullpointerexcept', 142), ('support_index_host', 141), ('inn

In [ ]:
#retaining 40 most common bigrams and trigrams in the corpus
bigramset = []   
trigramset = []
for i,j in zip(most_occur,most_occuring):
    bigramset.append(i[0])
    trigramset.append(j[0])
reg = '\w+\_\w+'
rege = '\w+\_\w+\_\w+'
REG = re.compile('[\_]')
for i in range(len(dataset)):
    st = ""
    for token in trigram[bigram[dataset[i]]]:
        if re.search(reg,token) and token not in bigramset :
            token = REG.sub(' ',token)
        elif re.search(rege,token) and token not in trigramset:
            token = REG.sub(' ',token)
        st += " "+token
    corpus[i] = st.strip() 

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
from collections import Counter
l = []
for i in range(len(corpus)): 
    l+=(corpus[i].split()) 
Counter1 = Counter(l)
most_occuring = Counter1.most_common(40) #calculates 40 common words in corpus
print(most_occuring)

[('geeksforgeek', 66171), ('link', 60668), ('use', 59399), ('1', 50983), ('articl', 44847), ('close', 37207), ('2', 35458), ('code', 33791), ('edit', 31431), ('write', 30894), ('number', 29674), ('output', 28677), ('exampl', 25273), ('main', 24334), ('help', 24200), ('method', 24086), ('element', 23935), ('appear', 22780), ('page', 22588), ('function', 21913), ('mail', 21878), ('org', 21820), ('return', 21147), ('program', 19659), ('button', 18188), ('3', 17870), ('0', 15354), ('time', 14680), ('list', 11620), ('4', 11251), ('string', 11225), ('approach', 11189), ('object', 10995), ('set', 10967), ('class', 10079), ('implement', 9733), ('array', 9722), ('type', 9512), ('5', 8727), ('index', 8653)]


In [ ]:
#removing 40 most common words from the corpus
freq_words = []
for i in most_occuring:
    freq_words.append(i[0])
for i in range(len(corpus)):
    corpus[i] = " ".join(words for words in corpus[i].split() if words not in freq_words)

Pre-processed data

In [ ]:
for i in range(0,11):
  print(corpus[i])
  print('\n')  

techniqu use opencv set 2 adapt threshold prerequisit simpl threshold use opencvin previous post simpl threshold explain differ type threshold techniqu threshold techniqu adapt threshold simpl threshold global valu threshold use remain constant constant threshold valu win t help case variabl light condit differ area adapt threshold method threshold valu calcul small region lead differ threshold valu differ region respect chang light use cv2 adaptivethreshold syntax adaptivethreshold sourc maxval adaptivemethod blocksiz constant paramet sourc input imag singl channel 8 bit float point maxval maximum valu assign adaptivemethod adapt method decid threshold valu calcul threshold valu mean neighbourhood area valu constant valu word mean neighborhood point constant threshold valu gaussian weight sum neighbourhood valu constant valu word weight sum neighborhood point constant thresholdtyp type threshold appli blocksiz size neighborhood use calcul threshold valu constant constant valu subtract

In [ ]:
#deallocating space for temporary variables
dataset,l1,l2,l = [],[],[],[]
most_occur,most_occuring = [],[] 

K-Means SK-Learn 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.cluster import KMeans 
from sklearn.metrics import silhouette_score 
import math,time 
vec = TfidfVectorizer() 
t0 = time.time()    
vector = vec.fit(corpus) 
X = vector.transform(corpus) 
if X.shape[0] > 5000: 
  k = math.ceil(X.shape[0]/5000) 
else: 
  k = 2 
km = KMeans(n_clusters = k,random_state=42) 
km.fit(X) 
labels = km.labels_ 
print(labels) 
t_km = time.time() - t0 
print(t_km) 

[0 2 0 ... 0 2 0]
496.4982361793518


Mini Batch sklearn

In [14]:
#collecting mini-batch objects for all 63 
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import math,time
from joblib import parallel_backend 
vec = TfidfVectorizer() 
mk_ob = []
t0 = time.time()
with parallel_backend('threading', n_jobs=2):
    X = vec.fit_transform(corpus)
    if X.shape[0] > 5000:
        k = math.ceil(X.shape[0]/5000)
    else:
        k = 2
    mk = MiniBatchKMeans(n_clusters = k,random_state = 42,batch_size = 5000,max_iter = 500)
    mk.fit(X)
mb_labels = mk.labels_
t_mini_batch = time.time() - t0
print(t_mini_batch)

NameError: ignored

In [ ]:
df = pd.read_csv("/content/cluster 1 project.csv" , encoding = "ISO-8859-1")

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
df.cluster.value_counts()

NameError: ignored

In [ ]:
def cos_similarity(textlist):
    tfidf = vec.fit_transform(textlist)
    return (tfidf * tfidf.T).toarray()

In [ ]:
label1=[]
label2=[]
label3=[]
label4=[]
label5=[]
for i in range(len(corpus)):
  if labels[i]==0:
    label1.append(i)
  elif labels[i]==1:
    label2.append(i)
  elif labels[i]==2:
    label3.append(i)
  elif labels[i]==3:
    label4.append(i)
  elif labels[i]==4:
    label5.append(i)
print(label1)
print(len(label1))
print(label2)
print(len(label2))
print(label3)
print(len(label3))
print(label4)
print(len(label4))
print(label5)
print(len(label5))

[0, 2, 3, 4, 7, 8, 9, 10, 13, 14, 15, 19, 20, 25, 27, 28, 29, 31, 36, 37, 38, 41, 44, 45, 49, 50, 52, 53, 56, 57, 59, 60, 63, 65, 66, 67, 70, 74, 75, 77, 78, 80, 81, 82, 84, 85, 86, 87, 90, 98, 101, 104, 107, 108, 111, 114, 115, 120, 125, 126, 128, 129, 133, 136, 137, 141, 145, 149, 151, 152, 153, 155, 160, 163, 164, 166, 167, 169, 170, 172, 173, 177, 178, 180, 182, 183, 186, 187, 191, 193, 195, 198, 199, 200, 203, 204, 209, 211, 212, 213, 217, 218, 222, 225, 228, 230, 232, 234, 238, 240, 242, 243, 248, 250, 251, 252, 254, 258, 261, 264, 266, 267, 268, 269, 272, 273, 276, 277, 278, 280, 281, 284, 285, 286, 292, 294, 295, 299, 301, 303, 308, 309, 310, 312, 313, 316, 317, 320, 321, 322, 323, 328, 333, 334, 335, 338, 343, 347, 349, 350, 352, 354, 356, 358, 359, 360, 361, 363, 364, 365, 366, 367, 371, 372, 379, 380, 383, 384, 386, 388, 390, 391, 398, 401, 403, 405, 406, 410, 412, 413, 414, 415, 418, 420, 422, 423, 424, 427, 428, 432, 434, 435, 437, 442, 443, 446, 447, 456, 459, 460, 462, 4

Cosine similarity values in a cluster

In [ ]:
clu_str = []
in_dict = {}
j = 0
for i in range(len(labels)):
  if labels[i] == 0:
    in_dict[j] = i
    clu_str += [corpus[i]]
    j += 1
ar = cos_similarity(clu_str)
print(ar)

[[1.         0.0597897  0.0391228  ... 0.02442067 0.0242035  0.01060162]
 [0.0597897  1.         0.00589709 ... 0.02486083 0.05483882 0.00230813]
 [0.0391228  0.00589709 1.         ... 0.14051168 0.02916933 0.03080091]
 ...
 [0.02442067 0.02486083 0.14051168 ... 1.         0.05109261 0.02809736]
 [0.0242035  0.05483882 0.02916933 ... 0.05109261 1.         0.01112932]
 [0.01060162 0.00230813 0.03080091 ... 0.02809736 0.01112932 1.        ]]


**Generating Recommendations**

In [ ]:
key_list = list(in_dict.keys()) 
val_list = list(in_dict.values())
for ind in range(0,2):
    related_docs_indices = ar[ind].argsort()[-1:-7:-1] #finding top 6 cosine similarities of that index
    #for j in range(len(related_docs_indices)):
    #  print(j)
    #print(related_docs_indices)
    for i in related_docs_indices:
      #print(in_dict[ind])
      print(ar[ind][i])    
      print(titletoind_dict[in_dict[i]])

1.0000000000000004
Python | Thresholding techniques using OpenCV | Set-2 (Adaptive Thresholding)
0.7922431366467444
Python | Thresholding techniques using OpenCV | Set-3 (Otsu Thresholding)
0.784191016696365
Python | Thresholding techniques using OpenCV | Set-1 (Simple Thresholding)
0.6487238144741531
OpenCV: Segmentation using Thresholding
0.6466646135536505
MATLAB | Converting a Grayscale Image to Binary Image using Thresholding
0.5931316067257338
Python | Pandas Series.clip_upper()
0.9999999999999998
C | Operators | Question 17
0.5076924173863067
C | Pointer Basics | Question 8
0.4779619222890549
C | Operators | Question 26
0.4544517226864807
C | Pointer Basics | Question 2
0.45268917852181856
C | Misc | Question 7
0.4318624291709071
C++ | Misc C++ | Question 2


In [ ]:
data["top1"]=""
data["top2"]=""
data["top3"]=""
data["top4"]=""
data["top5"]=""
data

,title,content,tags,top1,top2,top3,top4,top5
0,Python | Thresholding techniques using OpenCV ...,Prerequisite: Simple Thresholding using OpenC...,"{'Python', 'OpenCV', 'Image-Processing'}",,,,,
1,Generation of n numbers with given set of factors,"Given an array of k numbers factor[], the task...","{'prime-factor', 'Mathematical'}",,,,,
2,C | Operators | Question 17,Which of the following can have different mean...,"{'C', 'C-Operators', 'Operators', 'C Quiz'}",,,,,
3,SymPy | Subset.size() in Python,Subset.size() : size() is a sympy Python libra...,{'Python'},,,,,
4,Python PIL | ImageFont.truetype(),PIL is the Python Imaging Library which provid...,"{'Python', 'Python-pil'}",,,,,
...,...,...,...,...,...,...,...,...
24484,Importance of Hashcode method in Java,Prerequisite: Equals() and hashCode() methods ...,"{'Java', 'java-hashset', 'Java-HashMap'}",,,,,
24485,Ruby | Queue close() function,The close() is an inbuilt function in Ruby cl...,"{'Ruby Collections', 'Ruby'}",,,,,
24486,Scala | Field Overriding,"In any object-oriented programming language, O...","{'Scala', 'Scala-OOPS'}",,,,,
24487,Sum of minimum elements of all possible sub-ar...,"Given an array arr[], the task is to find the ...","{'C++ Programs', 'Data Structures', 'Stack', '...",,,,,


Importing recommendations into dataset

In [ ]:
key_list = list(in_dict.keys()) 
val_list = list(in_dict.values())
for ind in range(0,len(label5)):
    related_docs_indices = ar[ind].argsort()[-1:-7:-1] 
   # print(related_docs_indices)#finding top 6 cosine similarities of that index
    for j in range(len(related_docs_indices)):
      if j == 1:
        a=related_docs_indices[j]
        data.loc[in_dict[ind], 'top1'] = titletoind_dict[in_dict[a]] 
      elif j == 2:
        a=related_docs_indices[j]
        data.loc[in_dict[ind], 'top2'] = titletoind_dict[in_dict[a]] 
      elif j == 3:
        a=related_docs_indices[j]
        data.loc[in_dict[ind], 'top3'] = titletoind_dict[in_dict[a]] 
      elif j == 4:
        a=related_docs_indices[j]
        data.loc[in_dict[ind], 'top4'] = titletoind_dict[in_dict[a]] 
      elif j == 5:
        a=related_docs_indices[j]
        data.loc[in_dict[ind], 'top5'] = titletoind_dict[in_dict[a]] 

    
        #print(titletoind_dict[in_dict[a]])
    
      #data.loc[in_dict[ind], 'Top1'] = titletoind_dict[in_dict[ind]]    
data

,title,content,tags,top1,top2,top3,top4,top5
0,Python | Thresholding techniques using OpenCV ...,Prerequisite: Simple Thresholding using OpenC...,"{'Python', 'OpenCV', 'Image-Processing'}",Python | Thresholding techniques using OpenCV ...,Python | Thresholding techniques using OpenCV ...,OpenCV: Segmentation using Thresholding,MATLAB | Converting a Grayscale Image to Binar...,Python | Pandas Series.clip_upper()
1,Generation of n numbers with given set of factors,"Given an array of k numbers factor[], the task...","{'prime-factor', 'Mathematical'}",,,,,
2,C | Operators | Question 17,Which of the following can have different mean...,"{'C', 'C-Operators', 'Operators', 'C Quiz'}",C | Pointer Basics | Question 8,C | Operators | Question 26,C | Pointer Basics | Question 2,C | Misc | Question 7,C++ | Misc C++ | Question 2
3,SymPy | Subset.size() in Python,Subset.size() : size() is a sympy Python libra...,{'Python'},SymPy | Polyhedron.size() in Python,SymPy | Subset.prev_binary() in Python,SymPy | Subset.unrank_gray() in Python,SymPy | Subset.subset() in Python,Tug of War
4,Python PIL | ImageFont.truetype(),PIL is the Python Imaging Library which provid...,"{'Python', 'Python-pil'}",Python PIL | ImageFont.load_default(),How to set the Font of the Button in C#?,JavaFX | Font Class,How to set the font of the content present in ...,How to set the Font of the MaskedTextBox in C#?
...,...,...,...,...,...,...,...,...
24484,Importance of Hashcode method in Java,Prerequisite: Equals() and hashCode() methods ...,"{'Java', 'java-hashset', 'Java-HashMap'}",,,,,
24485,Ruby | Queue close() function,The close() is an inbuilt function in Ruby cl...,"{'Ruby Collections', 'Ruby'}",,,,,
24486,Scala | Field Overriding,"In any object-oriented programming language, O...","{'Scala', 'Scala-OOPS'}",,,,,
24487,Sum of minimum elements of all possible sub-ar...,"Given an array arr[], the task is to find the ...","{'C++ Programs', 'Data Structures', 'Stack', '...",,,,,


In [ ]:
data.to_csv('/content/drive/MyDrive/Recommendations.csv')

In [ ]:
#avg cosine similarities of each cluster for a particular board 
import statistics 
for l in range(5): 
    clu_str = []  
    for i in range(len(labels)): 
        if labels[i] == l: 
            clu_str += [corpus[i]] 
    ar = cos_similarity(clu_str) 
    avg = 0 
    for i in range(len(ar)): 
        ind = ar[i].argsort()[-2:-7:-1] 
        avg = avg + (statistics.mean(ar[i][ind][:5])) 
    print(avg/len(ar)) 

0.46573982040127815
0.41799066730306067
0.5171674929673996
0.48341903326098246
0.5787138404083364


In [ ]:
import statistics 
for l in range(5): 
    clu_str = [] 
    for i in range(len(mb_labels)): 
        if mb_labels[i] == l: 
            clu_str += [corpus[i]] 
    ar = cos_similarity(clu_str) 
    avg = 0 
    for i in range(len(ar)): 
        ind = ar[i].argsort()[-2:-7:-1] 
        avg = avg + (statistics.mean(ar[i][ind][:5])) 
    print(avg/len(ar)) 

0.5229362636870544
0.5027681750109002
0.48398418981283814
0.5014293986155416
0.5649332557020926


**Classification algorithm**

In [ ]:
from sklearn.model_selection import train_test_split
print("test shape : ", test.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error,mean_absolute_error, make_scorer,classification_report,confusion_matrix,accuracy_score,roc_auc_score,roc_curve

In [ ]:
tf_idf_vect = TfidfVectorizer(ngram_range=(0,5))
X_train = tf_idf_vect.fit_transform(train.content.values)
X_test = tf_idf_vect.transform(test.content.values)

y_train = train.cluster.values
y_test = test.cluster.values

print("X_train.shape : ", X_train.shape)
print("X_test.shape : ", X_test.shape)
print("y_train.shape : ", y_train.shape)
print("y_test.shape : ", y_test.shape)

X_train.shape :  (17142, 3153757)
X_test.shape :  (7347, 3153757)
y_train.shape :  (17142,)
y_test.shape :  (7347,)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_prediction = knn.predict(X_test)
knn_accuracy = accuracy_score(y_test,knn_prediction)
print("training accuracy Score    : ",knn.score(X_train,y_train))
print("testation accuracy Score : ",knn_accuracy )
print(classification_report(knn_prediction,y_test))

training accuracy Score    :  0.7993232994983083
testation accuracy Score :  0.7211106574111883
              precision    recall  f1-score   support

           0       0.26      0.84      0.39       101
           1       0.19      0.80      0.30        75
           2       0.33      0.79      0.46       813
           3       0.73      0.90      0.81       245
           4       0.97      0.70      0.81      6113

    accuracy                           0.72      7347
   macro avg       0.49      0.81      0.56      7347
weighted avg       0.87      0.72      0.76      7347



Gives cluster for every new data

In [ ]:
## copying two strings from our dataset
text_string=df['content'][5],df['content'][2],df['content'][3]
#text_string='generat n factor factor task print n ascend order factor problem main extens ugli problem creat auxiliari size factor track multipl factor iter minimum increment respect factor minimum equal previous increment avoid duplic print minimum n k auxiliari_space_o k comment_incorrect_want_share inform_topic_discuss','sum minimum possibl sub task sum minimum'
## Applying tfidf to two strings
vector=tf_idf_vect.transform(text_string)
## Getting the prediction
knn.predict(vector)

array([0, 4, 4])